# ყველა ფაილის მისამართის ამოღება

In [37]:
!pip install --upgrade librosa
!pip install --upgrade pydub
!pip install --upgrade pip

Requirement already up-to-date: librosa in /home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages (0.7.2)
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: pydub in /home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages (0.23.1)
You should consider upgrading via the 'pip install --upgrade pip' command.
     |██                              | 92kB 499kB/s eta 0:00:03ERROR: Exception:
Traceback (most recent call last):
  File "/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/pip/_vendor/urllib3/contrib/pyopenssl.py", line 313, in recv_into
    return self.connection.recv_into(*args, **kwargs)
  File "/home/tamar/.local/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1822, in recv_into
    self._raise_ssl_error(self._ssl, result)
  File "/home/tamar/.local/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1622, in _raise_ssl_error
    raise WantReadError()
OpenSSL.SSL.WantReadError

During handling of the above exce

In [2]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [3]:
import pandas as pd
import librosa
import librosa.display

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [4]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
        flux = get_spectral_features(audio, sample_rate)[2]
#         zero_crossing_rate = get_zero_crossing_rate(audio)
#         spectral_features = get_spectral_features(audio, sample_rate)
        chroma_features = get_chroma_features(audio, sample_rate)
        return [mfccs, flux] + chroma_features
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 

In [5]:
def get_all_features(folder):
    folders = os.listdir(folder)
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/' + folder +'/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'flux', 'chroma_features1', 'chroma_features2', 'chroma_features3', 'class_label'])
    return data

In [6]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=80)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, სხვადასხვა), mfccsscaled - (40,)
    return mfccs_scaled

In [7]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
    rate_scaled = np.mean(rate.T,axis=0)
#     rate - (1, სხვადასხვა)
    return rate_scaled

In [8]:
def get_spectral_features(audio, sr):
    sp = librosa.feature.spectral_centroid(audio)
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    onset_env.shape = (onset_env.shape[0], 1)
    sp_scaled = np.mean(sp.T,axis=0)
    a_scaled = np.mean(a.T,axis=0)
    onset_env_scaled = np.mean(onset_env,axis=0)
    return [sp_scaled, a_scaled, onset_env_scaled]

In [9]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, სხვადასხვა)
    chroma_stft_scaled = np.mean(chroma_stft.T,axis=0)
    chroma_cens_scaled = np.mean(chroma_cens.T,axis=0)
    chroma_cq_scaled = np.mean(chroma_cq.T,axis=0)
    return [chroma_stft_scaled, chroma_cens_scaled, chroma_cq_scaled]

In [10]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, სხვადასხვა)
    pitches_scaled = np.mean(pitches.T,axis=0)
    return pitches_scaled

In [11]:
data = get_all_features('data')

In [12]:
data = data.sample(frac=1).reset_index(drop=True)
data

,mfccs,flux,chroma_features1,chroma_features2,chroma_features3,class_label
0,"[-406.3137, 90.07318, -17.52547, 19.188358, -4...",[1.2665296],"[0.50325, 0.43390477, 0.45751518, 0.55981123, ...","[0.2010815228001731, 0.17877661978133974, 0.13...","[0.41260996, 0.35917467, 0.3394772, 0.4229124,...",3
1,"[-479.55746, 91.62016, 4.8111367, 45.4034, 7.1...",[2.0787861],"[0.710502, 0.6724909, 0.6182017, 0.53073376, 0...","[0.31813983391072426, 0.3531293189845657, 0.28...","[0.620216, 0.6951215, 0.61286265, 0.61554337, ...",1
2,"[-412.65436, 82.89575, -14.282551, -18.198072,...",[1.4700956],"[0.38203648, 0.32822415, 0.39516246, 0.3653528...","[0.2811999780220349, 0.3090476569723263, 0.313...","[0.5648398, 0.59924424, 0.63488513, 0.58711034...",3
3,"[-492.4508, 52.55787, -11.094436, 24.305426, -...",[1.2690605],"[0.35530308, 0.33277103, 0.35659152, 0.4583785...","[0.2933080292151442, 0.2664511925875559, 0.244...","[0.6027757, 0.57053393, 0.5303954, 0.58117336,...",3
4,"[-559.8948, 53.037304, 23.096941, 8.38059, 2.2...",[1.2796246],"[0.44477203, 0.36253828, 0.23909615, 0.2377628...","[0.38282826680034837, 0.3370629763078627, 0.25...","[0.6155926, 0.56290025, 0.47744942, 0.41551587...",3
...,...,...,...,...,...,...
683,"[-488.35214, 63.34991, 18.823492, 17.853516, -...",[0.9126781],"[0.39689508, 0.3515242, 0.34030342, 0.25555554...","[0.365042918727885, 0.3005016817024467, 0.2271...","[0.7129718, 0.5999869, 0.49596426, 0.4806125, ...",5
684,"[-647.4395, 53.12541, 14.57684, 29.460068, -3....",[1.3850218],"[0.62824255, 0.47224903, 0.32193753, 0.2552542...","[0.2849922771460006, 0.24627764286685888, 0.20...","[0.51113725, 0.50479937, 0.39885342, 0.5197480...",4
685,"[-679.07196, 84.32127, 1.3979447, 10.759451, -...",[0.9670585],"[0.40746984, 0.45578644, 0.47212955, 0.4500661...","[0.29454657735332035, 0.27965101813545123, 0.2...","[0.61592126, 0.5651262, 0.5621043, 0.5231517, ...",3
686,"[-481.00757, 79.54055, 0.37326252, 19.646515, ...",[1.3181589],"[0.31452677, 0.2739423, 0.29177672, 0.23973174...","[0.31061829776123817, 0.2660416390428546, 0.27...","[0.62170583, 0.5375296, 0.56233233, 0.54323864...",4


In [13]:
from sklearn.model_selection import train_test_split 
labels = [[i] for i in data['class_label']]
# testLabels = [[i] for i in testData['class_label']]
y_train = np.array(labels)
# y_test = np.array(testLabels)

# x_train, x_test, y_train, y_test = train_test_split(data, np.array(labels), test_size=0.1, random_state = 127)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# One-hot encode

In [14]:
from sklearn.preprocessing import OneHotEncoder
def get_one_hot(y):
    encoder = OneHotEncoder(sparse=False)
    y_onehot = encoder.fit_transform(y)
    return y_onehot

In [15]:
y_onehot_train = get_one_hot(y_train)
# y_onehot_test = get_one_hot(y_test)
# print(y_onehot_train.shape, y_onehot_test.shape)

/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# PCA

In [16]:
def pca(X):
    # normalize the features
    X = (X - X.mean()) / X.std()
    
    # compute the covariance matrix
    X = np.matrix(X)
    cov = (X.T * X) / X.shape[0]
    
    # perform SVD
    U, S, V = np.linalg.svd(cov)
    
    return U, S, V

In [17]:
def project_data(X, U, k):
    U_reduced = U[:,:k]
    return np.dot(X, U_reduced)

In [18]:
def recover_data(Z, U, k):
    U_reduced = U[:,:k]
    return np.dot(Z, U_reduced.T)

# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [19]:
def get_processed_X(x):
    X = x.drop(columns=['class_label'])
    X = X.values
    ls = [] #featurebi gvinda da magitom
    for i in range(X.shape[0]):
        features = []
        for j in range(X.shape[1]):
            for k in X[i][j]:
                features.append(k)
        ls.append(features)
    res = np.array(ls)
    return res

In [20]:
ls_train = get_processed_X(data)
# ls_test = get_processed_X(x_test)
# print(ls_train.shape, ls_test.shape)

In [21]:
k = 400
U, S, V = pca(ls_train)
ls_train = project_data(ls_train, U, k)
# U1, S1, V1 = pca(ls_test)
# ls_test = project_data(ls_test, U, k)

 # ნეირონული ქსელის კოდი

In [22]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [23]:
import math

def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]

In [24]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = []
    for i in range(z3.shape[0]):
        z = [z3[i, j] for j in range(z3.shape[1])]
        h.append(softmax(z))
    h = np.array(h)
#     h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [25]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

#     J = (-1 / m) * np.sum(y * np.log(h).T) #softmax
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [26]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [27]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    J = cost(params,input_size, hidden_size, num_labels, X, y, learning_rate)

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [28]:
# initial setup
input_size = ls_train.shape[1]
hidden_size = 25
num_labels = y_onehot_train.shape[1]
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls_train.shape[0]
ls_train = np.matrix(ls_train)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((25, 118), (5, 26))

In [29]:
a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape
print(h)
print(cost(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate))

[[0.17247922 0.28789608 0.14883344 0.12546897 0.2653223 ]
 [0.17241979 0.28794842 0.14883973 0.12550678 0.26528528]
 [0.17192533 0.28863002 0.14884698 0.12585387 0.2647438 ]
 ...
 [0.16935671 0.29224183 0.14871209 0.12761853 0.26207085]
 [0.17227469 0.28818    0.148826   0.12560841 0.26511089]
 [0.17247871 0.28789585 0.14883471 0.12547132 0.2653194 ]]
2.5771617268364184


In [30]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate)
J, grad.shape

(2.5771617268364184, (3080,))

In [31]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 500})
fmin

     fun: 1.0892041513527524
     jac: array([-0.00069153, -0.4754602 , -0.04605256, ..., -0.00281459,
       -0.00148521,  0.00168297])
 message: 'Max. number of function evaluations reached'
    nfev: 500
     nit: 64
  status: 3
 success: False
       x: array([-0.10506967,  0.00430652,  0.09777166, ...,  0.52999959,
        0.3128878 , -0.53126736])

In [32]:
ls_train = np.matrix(ls_train)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
y_pred_train = np.array(np.argmax(h, axis=1) + 1)

In [33]:
thetas = []
thetas.append(theta1)
thetas.append(theta2)
np.save('weights', thetas)

In [34]:
# ls_test = np.matrix(ls_test)

# a1, z2, a2, z3, h = forward_propagate(ls_test, theta1, theta2)
# y_pred = np.array(np.argmax(h, axis=1) + 1)

In [35]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred_train, y_train)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.8284883720930233


In [36]:
# correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
# accuracy = (sum(map(int, correct)) / float(len(correct)))
# print(accuracy)